In [2]:
import pandas as pd
from pandas_datareader import data as pdr
from datetime import datetime
nifty_tickers=pd.read_csv('Nifty_yahoo_ticker.csv')
nifty_list=nifty_tickers['Yahoo_Symbol'].tolist()
end = datetime.today()
begin=end-pd.DateOffset(5) #Change this number
st=begin.strftime('%Y-%m-%d') 
ed=end.strftime('%Y-%m-%d')
data=[]
niftyd_list=[]
for i,k in enumerate(nifty_list):
    try:
        data.append(pdr.get_data_yahoo(k,st,ed))
        niftyd_list.append(k)
        
    except Exception:
        print('Not found',k)
        
        pass
result={}
writer=pd.ExcelWriter("Nifty50_Datanew.xlsx")
for i, A in enumerate(data):
    A.to_excel(writer,sheet_name="{0}".format(niftyd_list[i]))
writer.save()
data_old=pd.read_excel("Nifty50_Data.xlsx",sheet_name=None,index_col=0)
data_new=pd.read_excel("Nifty50_Datanew.xlsx",sheet_name=None,index_col=0)
for key in (data_old.keys() | data_new.keys()):
    if key in data_old: result.setdefault(key, []).append(data_old[key])
    if key in data_new: result.setdefault(key, []).append(data_new[key])
def convert_dict_to_list(result):
    name_list=[]
    df_list=[]
    for key in result.keys():
        name_list.append(key)
        
    for name in name_list:
        kk=result[name]
        kf=pd.concat(kk,axis=0)
        df3 = kf.loc[~kf.index.duplicated(keep='first')]
        df3.name=name
        df_list.append(df3)
        
    return df_list
df_list=[]
df_list=convert_dict_to_list(result)
writer=pd.ExcelWriter("Nifty50_Data.xlsx")
for i, A in enumerate(df_list):
    A.to_excel(writer,sheet_name="{0}".format(df_list[i].name))
writer.save()